# sprint13 ディープラーニング フレームワーク1


## 1.このSprintについて

### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

### どのように学ぶか
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

## 2.コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。



### 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）

- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。



【回答】<br>
- 学習率の設定
- バイアスと重みの初期値設定
- 活性化関数
- バッチサイズの指定
- エポックのループ
- バイアスと重みの更新

データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


Iris Species


目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。<br>
Iris-versicolor<br>
Iris-virginica<br>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


《サンプルコード》


＊バージョン1.5から1.14の間で動作を確認済みです。

In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Colocations handled automatically by placer.
Epoch 0, loss : 161.1374, val_loss : 70.3554, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 28.6872, val_loss : 70.7311, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 53.0105, val_loss : 23.3513, acc : 0.250, val_acc : 0.375
Epoch 3, loss : 9.7703, val_loss : 23.4089, acc : 0.250, val_acc : 0.312
Epoch 4, loss : 16.8688, val_loss : 9.8314, acc : 0.250, val_acc : 0.188
Epoch 5, loss : 11.4232, val_loss : 6.2517, acc : 0.250, val_acc : 0.500
Epoch 6, loss : 7.9160, val_loss : 5.1624, acc : 0.250, val_acc : 0.625
Epoch 7, loss : 1.7044, val_loss : 2.2781, acc : 0.750, val_acc : 0.688
Epoch 8, loss : 0.0000, val_loss : 0.5030, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.2250, acc : 1.000, val_acc : 0.812
test_acc : 0.850


【回答】
- 学習率の設定→66行目learning_rate = 0.01
- バイアスと重みの初期値設定→83行目〜93行目
- 活性化関数→95行目、97行目
- バッチサイズの指定→67行目
- エポックのループ→117行目
- バイアスと重みの更新→

## 3.他のデータセットへの適用
これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


Iris（3種類全ての目的変数を使用）
House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))<br>
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))<br>

メソッドは以下のように公式ドキュメントを確認してください。


tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow


tf.math.sign  |  TensorFlow


＊tf.signとtf.math.signは同じ働きをします。


In [3]:
from sklearn.preprocessing import OneHotEncoder
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

In [4]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])

In [5]:
y_one_hot.shape

(150, 3)

In [6]:
y_one_hot = y_one_hot.astype(np.int)

In [7]:
y_one_hot.shape

(150, 3)

In [8]:

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [9]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [10]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [11]:
tf.math.sign([0.2, 0.7, 0.1])

<tf.Tensor 'Sign_2:0' shape=(3,) dtype=float32>

In [12]:
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.0000, val_loss : 5.1974, acc : 1.000, val_acc : 0.583
Epoch 1, loss : 0.0000, val_loss : 4.3749, acc : 1.000, val_acc : 0.708
Epoch 2, loss : 0.0000, val_loss : 2.3456, acc : 1.000, val_acc : 0.917
Epoch 3, loss : 0.0304, val_loss : 4.5126, acc : 1.000, val_acc : 0.792
Epoch 4, loss : 0.0000, val_loss : 0.4803, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 2.2069, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 0.7208, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0011, val_loss : 3.6810, acc : 1.000, val_acc : 0.833
Epoch 8, loss : 0.0000, val_loss : 2.4646, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0000, val_loss : 3.0435, acc : 1.000, val_acc : 0.917
test_acc : 0.967


## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [14]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)

In [15]:
# 対数変換
y = np.log(y)

In [16]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [18]:
# ハイパーパラメータの設定
learning_rate = 1e-7
batch_size = 10
num_epochs = 10
# n_hidden1 = 50
# n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [19]:
# 係数（W）と定数項（b）の変数
W = tf.Variable(tf.random_normal([n_input,1]))
b = tf.Variable(tf.random_normal([n_classes]))

In [20]:
# 線形モデル
yp = tf.add(b, tf.matmul(X, W))
# 目的関数
loss_op = tf.reduce_mean(tf.square(yp - Y))
# 最適化手法
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = yp
# 指標値計算
accuracy = tf.reduce_mean(tf.square(correct_pred - Y))
# variableの初期化
init = tf.global_variables_initializer()

In [21]:
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 25.5109, val_loss : 42.8527, acc : 25.511, val_acc : 42.853
Epoch 1, loss : 0.2160, val_loss : 0.2336, acc : 0.216, val_acc : 0.234
Epoch 2, loss : 0.0685, val_loss : 0.1194, acc : 0.068, val_acc : 0.119
Epoch 3, loss : 0.0639, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 4, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 5, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 6, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 7, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 8, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
Epoch 9, loss : 0.0637, val_loss : 0.1190, acc : 0.064, val_acc : 0.119
test_acc : 0.150


## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [29]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


### tensorbordに出力してみる

In [32]:
# 入力データを定義
x = tf.placeholder(tf.float32, [None, 784])

# 入力画像をログに出力
img = tf.reshape(x,[-1, 28, 28, 1])
tf.summary.image("input_data", img, 10)    # 10枚表示

# ハイパーパラメータの設定
learning_rate = 0.5
batch_size = 10
num_epochs = 10
n_hidden1 = 64
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 初期化 正規分布 標準偏差0.1
tf.truncated_normal([n_input, n_hidden1], stddev=0.1)
tf.matmul(x, w_1)

# 入力層から中間層
with tf.name_scope("hidden"):
    w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
    b_1 = tf.Variable(tf.zeros([64]), name="b1")
    h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)
    
    # 中間層の重みの分布をログ出力
    tf.summary.histogram("w_1", w_1)

# 中間層から出力層
with tf.name_scope("output"):
    w_2 = tf.Variable(tf.truncated_normal([n_hidden1, n_classes], stddev=0.1), name="w2")
    b_2 = tf.Variable(tf.zeros([n_classes]), name="b2")
    out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)
    

y = tf.placeholder(tf.float32, [None, n_classes]) 

#loss_op = tf.reduce_mean(tf.square(y - out))
# 損失関数
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y - out))

# train
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# train_op = optimizer.minimize(loss_op)
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
# 評価
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# 誤差をログ出力
tf.summary.scalar("loss", loss)

# 精度をログ出力
tf.summary.scalar("accuracy", accuracy)

# 全てのログをマージ
summary_op = tf.summary.merge_all()
# 初期化
init = tf.global_variables_initializer()

In [33]:
with tf.Session() as sess:
    sess.run(init)
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(500):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
        summary_writer = tf.summary.FileWriter("logs", sess.graph)
        
        if step % 10 == 0:
            acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            print("step %d:acc = %.2f " %(step, acc_val))
            # ログを取る処理を実行（出力はプロトコルバッファ）
            summary_str = sess.run(summary_op, feed_dict={x:test_images, y:test_labels})
            # ログ情報のプロトコルバッファを書き込む
            summary_writer.add_summary(summary_str, step)

step 10:acc = 0.17 
step 20:acc = 0.22 
step 30:acc = 0.27 
step 40:acc = 0.31 
step 50:acc = 0.32 
step 60:acc = 0.36 
step 70:acc = 0.38 
step 80:acc = 0.44 
step 90:acc = 0.47 
step 100:acc = 0.49 
step 110:acc = 0.53 
step 120:acc = 0.55 
step 130:acc = 0.58 
step 140:acc = 0.60 
step 150:acc = 0.62 
step 160:acc = 0.63 
step 170:acc = 0.64 
step 180:acc = 0.66 
step 190:acc = 0.68 
step 200:acc = 0.69 
step 210:acc = 0.70 
step 220:acc = 0.70 
step 230:acc = 0.72 
step 240:acc = 0.73 
step 250:acc = 0.72 
step 260:acc = 0.74 
step 270:acc = 0.74 
step 280:acc = 0.75 
step 290:acc = 0.76 
step 300:acc = 0.77 
step 310:acc = 0.78 
step 320:acc = 0.79 
step 330:acc = 0.79 
step 340:acc = 0.80 
step 350:acc = 0.81 
step 360:acc = 0.80 
step 370:acc = 0.82 
step 380:acc = 0.82 
step 390:acc = 0.83 
step 400:acc = 0.83 
step 410:acc = 0.84 
step 420:acc = 0.84 
step 430:acc = 0.84 
step 440:acc = 0.84 
step 450:acc = 0.84 
step 460:acc = 0.85 
step 470:acc = 0.85 
step 480:acc = 0.85 
s

tensorbordで確認することができた

### 畳み込み層、プーリング層、FC層の形で学習

In [36]:
# 入力データを定義
x = tf.placeholder(tf.float32, [None, 784])

# (バッチサイズ、高さ、幅、チャンネル数)に変更　入力画像をログに出力 tensorbord用
img = tf.reshape(x,[-1, 28, 28, 1])
#tf.summary.image("input_data", img, 10)    # tensorbord10枚表示

# 畳み込み層1
f1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
conv1 = tf.nn.conv2d(img, f1, strides=[1, 1, 1, 1], padding='SAME')    #stride1なら同じサイズになるようpaddingする
b1 = tf.Variable(tf.constant(0.1, shape=[32]))
h_conv1 = tf.nn.relu(conv1+b1)

# プーリング層1    N,28,28,1→N,14,14,32
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#畳み込み層2
f2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
conv2 = tf.nn.conv2d(h_pool1, f2, strides=[1, 1, 1, 1], padding='SAME')
b2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(conv2+b2)

# プーリング層2    N,14,14,32→N,7,7,64
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# flat
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# FC層
w_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc1= tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc1 = tf.matmul(h_pool2_flat, w_fc1) + b_fc1

# 出力層
w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
out = tf.nn.softmax(tf.matmul(h_fc1, w_fc2) + b_fc2)

# 交差エントロピー
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(out + 1e-5), axis=[1]))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 評価
correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# 初期化
init = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    sess.run(init)
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(500):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
#         summary_writer = tf.summary.FileWriter("logs", sess.graph)
        
        if step % 100 == 0:
            acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            print("step %d:acc = %.2f " %(step, acc_val))
#             # ログを取る処理を実行（出力はプロトコルバッファ）
#             summary_str = sess.run(summary_op, feed_dict={x:test_images, y:test_labels})
#             # ログ情報のプロトコルバッファを書き込む
#             summary_writer.add_summary(summary_str, step)

step 100:acc = 0.69 
step 200:acc = 0.88 
step 300:acc = 0.92 
step 400:acc = 0.94 
step 500:acc = 0.95 
